In [187]:
import pdftotext
import requests
import io
import re
import pandas as pd
import numpy as np
import json
import shutil
import math

In [11]:
with open('all_pdfs.txt') as pdfs:
    all_pdfs = json.load(pdfs)

In [157]:
small_pdfs2 = dict(list(all_pdfs.items())[10:13])

In [208]:
def create_corpus(all_pdfs): 
    pdf_urls = [[k, v] for k, v in all_pdfs.items()]
    for i, pdf_url in enumerate(pdf_urls):
        req = requests.get(pdf_url[1])
        with io.BytesIO(req.content) as f:
            pdf = pdftotext.PDF(f)
        text = ''
        for page in pdf:
            text += page
        pdf_url.append(text.strip())
        print('rank 0:', str((round((i + 1)/len(pdf_url)*100, 4))) + '%')
    return pd.DataFrame(pdf_urls, columns=['Resolution', 'url', 'Text'])

In [209]:
df = create_corpus(small_pdfs2)

rank 0: 33.3333%
rank 0: 66.6667%
rank 0: 100.0%


In [210]:
df

,Resolution,url,Text
0,A/RES/42/159,http://digitallibrary.un.org/record/152637/fil...,
1,A/RES/65/244,http://digitallibrary.un.org/record/698581/fil...,United Nations ...
2,A/RES/2423(XXIII),http://digitallibrary.un.org/record/202690/fil...,


In [183]:
s = df.at[1, 'Text']

In [165]:
LicenseCode = '8A97D470-81F1-4E8C-A464-81039D705658'
UserName =  'egemenpamukcu'
RequestUrl = "http://www.ocrwebservice.com/restservices/processDocument?gettext=true"
req = requests.get('http://digitallibrary.un.org/record/152637/files/A_RES_42_159-EN.pdf')
r = requests.post(RequestUrl, data=req.content, auth=(UserName, LicenseCode))
jobj = json.loads(r.content)
jobj["OCRText"][0]

[['IX. Resolutions adopted on the reports of the Sixth Committee 299 Bearing in mind that, for various reasons, there are par-ticularly favourable opportunities for co-operation and mutual advantage between neighbouring countries in many fields and various forms, and that the development of such co-operation may have a positive influence on in-ternational relations as a whole, Considering that the great changes of a political, eco-nomic and social nature, as well as the scientific and tech-nological advances that have taken place in the world and led to unprecedented interdependence of nations, have given new dimensions to good-neighbourliness in the con-duct of States and increased the need to develop and strengthen it, Taking into account the working papers concerning the development and strengthening of good-neighbourliness between States, as well as the written replies sent by States and international organizations on the content of good-neighbourliness and on ways and means to enh

In [177]:
from nltk import word_tokenize

In [185]:
all_pdfs

{'A/RES/2537(XXIV)[C]': 'http://digitallibrary.un.org/record/645837/files/A_RES_2537%28XXIV%29%5BC%5D-EN.pdf',
 'A/RES/60/98': 'http://digitallibrary.un.org/record/562730/files/A_RES_60_98-EN.pdf',
 'A/RES/44/154': 'http://digitallibrary.un.org/record/82474/files/A_RES_44_154-EN.pdf',
 'A/RES/69/219': 'http://digitallibrary.un.org/record/786724/files/A_RES_69_219-EN.pdf',
 'A/RES/52/108': 'http://digitallibrary.un.org/record/251370/files/A_RES_52_108-EN.pdf',
 'A/RES/45/54': 'http://digitallibrary.un.org/record/105310/files/A_RES_45_54-EN.pdf',
 'A/RES/74/102': 'http://digitallibrary.un.org/record/3846809/files/A_RES_74_102-EN.pdf',
 'A/RES/51/48': 'http://digitallibrary.un.org/record/230524/files/A_RES_51_48-EN.pdf',
 'A/RES/73/160': 'http://digitallibrary.un.org/record/1660233/files/A_RES_73_160-EN.pdf',
 'A/RES/1942(XVIII)': 'http://digitallibrary.un.org/record/203987/files/A_RES_1942%28XVIII%29-EN.pdf',
 'A/RES/42/159': 'http://digitallibrary.un.org/record/152637/files/A_RES_42_159

In [189]:
pdf_urls = [[k, v] for k, v in all_pdfs.items()]

n = math.ceil(len(pdf_urls) / 8)
partitions = [pdf_urls[i * n:(i + 1) * n] for i in range((len(pdf_urls) + n - 1) // n )]

In [190]:
len(partitions)

8

In [197]:
df.to_json('rcj{}.json'.format(0))

In [241]:
test = []
for i in range(8):
    if i not in (3, 5):
        test.append(pd.read_json('raw_corpus_pre{}.json'.format(i)))
test_df = pd.concat(test, ignore_index=True)

In [255]:
print(test_df[test_df.Text != ''].sample(n=1).iat[0, 2])

United Nations                                                                A/RES/62/255

                                                                                                     Distr.: General
                  General Assembly                                                                    22 July 2008




 Sixty-second session
 Agenda item 143


              Resolution adopted by the General Assembly on 20 June 2008
                                     [on the report of the Fifth Committee (A/62/868)]


              62/255. Financing of the United Nations Peacekeeping Force
                      in Cyprus

                    The General Assembly,
                   Having considered the reports of the Secretary-General on the financing of the
              United Nations Peacekeeping Force in Cyprus, 1 the note by the Secretary-General
                                                                    0F




              on the financing arrangements for the F

In [256]:
test_df

,Resolution,url,Text
0,A/RES/2537(XXIV)[C],http://digitallibrary.un.org/record/645837/fil...,
1,A/RES/60/98,http://digitallibrary.un.org/record/562730/fil...,United Nations ...
2,A/RES/44/154,http://digitallibrary.un.org/record/82474/file...,
3,A/RES/69/219,http://digitallibrary.un.org/record/786724/fil...,United Nations ...
4,A/RES/52/108,http://digitallibrary.un.org/record/251370/fil...,UNITED ...
...,...,...,...
12736,A/RES/3392(XXX),http://digitallibrary.un.org/record/189792/fil...,
12737,A/RES/35/54,http://digitallibrary.un.org/record/17247/file...,
12738,A/RES/37/83,http://digitallibrary.un.org/record/40878/file...,
12739,A/RES/60/18A,http://digitallibrary.un.org/record/561540/fil...,United Nations ...


In [ ]:
all_pdfs

In [257]:
pdf_urls = [[k, v] for k, v in all_pdfs.items()]

n = math.ceil(len(pdf_urls) / 8)
partitions = [pdf_urls[i * n:(i + 1) * n] for i in range((len(pdf_urls) + n - 1) // n )]

In [259]:
missing = partitions[3] + partitions[5]

In [263]:
missing_res = [i for i, v in missing]

In [266]:
missing_res

['A/RES/2663(XXV)[A]',
 'A/RES/2631(XXV)',
 'A/RES/36/188',
 'A/RES/40/83',
 'A/RES/56/72[B-X]',
 'A/RES/64/224',
 'A/RES/419(V)',
 'A/RES/65/65',
 'A/RES/38/5',
 'A/RES/62/233B',
 'A/RES/66/32',
 'A/RES/36/211',
 'A/RES/52/20',
 'A/RES/66/250',
 'A/RES/60/182',
 'A/RES/58/108[B]',
 'A/RES/3118(XXVIII)',
 'A/RES/229(III)',
 'A/RES/73/338',
 'A/RES/65/254A',
 'A/RES/3024(XXVII)',
 'A/RES/31/21',
 'A/RES/45/268',
 'A/RES/43/166',
 'A/RES/36/89',
 'A/RES/37/80',
 'A/RES/46/81',
 'A/RES/43/180',
 'A/RES/48/13B',
 'A/RES/55/121',
 'A/RES/59/270',
 'A/RES/70/294',
 'A/RES/2440(XXIII)',
 'A/RES/74/307',
 'A/RES/43/79',
 'A/RES/72/82',
 'A/RES/35/134',
 'A/RES/1609(XV)',
 'A/RES/36/24',
 'A/RES/3053(XXVIII)[F]',
 'A/RES/54/178',
 'A/RES/56/201',
 'A/RES/2243(XXI)',
 'A/RES/39/102',
 'A/RES/45/118',
 'A/RES/47/225',
 'A/RES/51/243',
 'A/RES/55/22',
 'A/RES/61/123',
 'A/RES/52/84',
 'A/RES/36/233',
 'A/RES/65/250',
 'A/RES/49/24',
 'A/RES/2431(XXIII)',
 'A/RES/51/153B',
 'A/RES/61/248B',
 'A/RES

In [281]:
with open('errored_pdfs.txt', 'w') as err_pdf:
    json.dump(dict(missing), err_pdf)

In [284]:
missing

[['A/RES/2663(XXV)[A]',
  'http://digitallibrary.un.org/record/645536/files/A_RES_2663%28XXV%29%5BA%5D-EN.pdf'],
 ['A/RES/2631(XXV)',
  'http://digitallibrary.un.org/record/201946/files/A_RES_2631%28XXV%29-EN.pdf'],
 ['A/RES/36/188',
  'http://digitallibrary.un.org/record/31532/files/A_RES_36_188-EN.pdf'],
 ['A/RES/40/83',
  'http://digitallibrary.un.org/record/108810/files/A_RES_40_83-EN.pdf'],
 ['A/RES/56/72[B-X]',
  'http://digitallibrary.un.org/record/459617/files/A_RES_56_72%5EB-X%5E-EN.pdf'],
 ['A/RES/64/224',
  'http://digitallibrary.un.org/record/673730/files/A_RES_64_224-EN.pdf'],
 ['A/RES/419(V)',
  'http://digitallibrary.un.org/record/210555/files/A_RES_419%28V%29-EN.pdf'],
 ['A/RES/65/65',
  'http://digitallibrary.un.org/record/696986/files/A_RES_65_65-EN.pdf'],
 ['A/RES/38/5',
  'http://digitallibrary.un.org/record/195786/files/A_RES_38_5-EN.pdf'],
 ['A/RES/62/233B',
  'http://digitallibrary.un.org/record/633097/files/A_RES_62_233B-EN.pdf'],
 ['A/RES/66/32',
  'http://digi

In [285]:
n = math.ceil(len(missing) / 8)
partitions = [missing[i * n:(i + 1) * n] for i in range((len(missing) + n - 1) // n )]

In [287]:
len(partitions[0])

531

In [288]:
test_err = []
for i in range(8):
    if i != 5:
        test_err.append(pd.read_json('raw_corpus_error{}.json'.format(i)))
test_df_err = pd.concat(test_err, ignore_index=True)

In [293]:
test_df_err

,Resolution,url,Text
0,A/RES/2663(XXV)[A],http://digitallibrary.un.org/record/645536/fil...,
1,A/RES/2631(XXV),http://digitallibrary.un.org/record/201946/fil...,
2,A/RES/36/188,http://digitallibrary.un.org/record/31532/file...,
3,A/RES/40/83,http://digitallibrary.un.org/record/108810/fil...,
4,A/RES/56/72[B-X],http://digitallibrary.un.org/record/459617/fil...,United Nations ...
...,...,...,...
3712,A/RES/1809(XVII),http://digitallibrary.un.org/record/204356/fil...,
3713,A/RES/54/75,http://digitallibrary.un.org/record/403877/fil...,UNITED ...
3714,A/RES/55/237,http://digitallibrary.un.org/record/430654/fil...,United Nations ...
3715,A/RES/50/147,http://digitallibrary.un.org/record/205547/fil...,UNITED ...


In [308]:
with open('errored_pdfs_2.txt', 'w') as err_pdf:
    json.dump(missing_2, err_pdf)

In [305]:
missing_2 = dict(partitions[5])

In [310]:
with open('all_pdfs.txt') as pds:
    all_pdfs = json.load(pds)

In [315]:
print(all_pdfs['A/RES/65/57'])

http://digitallibrary.un.org/record/697747/files/A_RES_65_57-EN.pdf


In [316]:
test_err_2 = []
for i in range(8):
    test_err_2.append(pd.read_json('raw_corpus_error_two_{}.json'.format(i)))
test_df_err_2 = pd.concat(test_err_2, ignore_index=True)

In [322]:
test_df_err_2 = test_df_err_2.drop(0)

In [327]:
resolutions_raw = pd.concat([test_df_err_2, test_df, test_df_err], ignore_index=True)

In [378]:
resolutions_raw.to_json('raw_resolutions.json')

In [386]:
raw = pd.read_json('raw_resolutions.json')

In [385]:
dict(zip(raw[raw.Text == ''].Resolution, raw[raw.Text == ''].url))

{'A/RES/1187(XII)': 'http://digitallibrary.un.org/record/207452/files/A_RES_1187%28XII%29-EN.pdf',
 'A/RES/34/233[V]': 'http://digitallibrary.un.org/record/615802/files/A_RES_34_233%5BV%5D-EN.pdf',
 'A/RES/32/62': 'http://digitallibrary.un.org/record/188355/files/A_RES_32_62-EN.pdf',
 'A/RES/37/131': 'http://digitallibrary.un.org/record/41899/files/A_RES_37_131-EN.pdf',
 'A/RES/323(IV)': 'http://digitallibrary.un.org/record/210337/files/A_RES_323%28IV%29-EN.pdf',
 'A/RES/3261(XXIX)[F]': 'http://digitallibrary.un.org/record/640868/files/A_RES_3261%28XXIX%29%5BF%5D-EN.pdf',
 'A/RES/2574(XXIV)[B]': 'http://digitallibrary.un.org/record/645858/files/A_RES_2574%28XXIV%29%5BB%5D-EN.pdf',
 'A/RES/32/87[B]': 'http://digitallibrary.un.org/record/623222/files/A_RES_32_87%5BB%5D-EN.pdf',
 'A/RES/43/72': 'http://digitallibrary.un.org/record/54152/files/A_RES_43_72-EN.pdf',
 'A/RES/1898(XVIII)': 'http://digitallibrary.un.org/record/203752/files/A_RES_1898%28XVIII%29-EN.pdf',
 'A/RES/44/79': 'http://

In [392]:
raw.iat[0, 1]

'http://digitallibrary.un.org/record/207452/files/A_RES_1187%28XII%29-EN.pdf'

In [399]:
import cv2
import pytesseract

In [ ]:
img = cv2.imread('/Users/user1/Desktop/folder1/pdf1.pdf')
text = pytesseract.image_to_string(img)
print(text)

In [391]:
import pdf2image

In [ ]:
pages=convert_from_path("pdf_file_to_convert")
for page in pages:
    page.save("page_image.jpg","jpg")

In [410]:
for 
    req = requests.get(url)
    pages = pdf2image.convert_from_bytes(req.content)
    res = ''
    for page in pages:
        res +=pytesseract.image_to_string(page)

In [427]:
for i, k in raw.iterrows():
    

http://digitallibrary.un.org/record/207452/files/A_RES_1187%28XII%29-EN.pdf
http://digitallibrary.un.org/record/853929/files/A_RES_71_100-EN.pdf
http://digitallibrary.un.org/record/454055/files/A_RES_56_48-EN.pdf
http://digitallibrary.un.org/record/458680/files/A_RES_56_254%5EB%5E-EN.pdf
http://digitallibrary.un.org/record/509653/files/A_RES_58_229-EN.pdf
http://digitallibrary.un.org/record/615802/files/A_RES_34_233%5BV%5D-EN.pdf
https://digitallibrary.un.org/record/3895292/files/A_RES_75_58-EN.pdf
http://digitallibrary.un.org/record/188355/files/A_RES_32_62-EN.pdf
http://digitallibrary.un.org/record/697255/files/A_RES_65_94-EN.pdf
http://digitallibrary.un.org/record/41899/files/A_RES_37_131-EN.pdf
http://digitallibrary.un.org/record/777805/files/A_RES_68_296-EN.pdf
http://digitallibrary.un.org/record/403875/files/A_RES_54_73-EN.pdf
http://digitallibrary.un.org/record/210337/files/A_RES_323%28IV%29-EN.pdf
http://digitallibrary.un.org/record/640868/files/A_RES_3261%28XXIX%29%5BF%5D-EN.p

http://digitallibrary.un.org/record/59143/files/A_RES_38_214-EN.pdf
http://digitallibrary.un.org/record/640888/files/A_RES_3358%28XXIX%29%5BB%5D-EN.pdf
http://digitallibrary.un.org/record/687201/files/A_RES_64_293-EN.pdf
http://digitallibrary.un.org/record/203962/files/A_RES_1909%28XVIII%29-EN.pdf
http://digitallibrary.un.org/record/204269/files/A_RES_1762%28XVII%29-EN.pdf
http://digitallibrary.un.org/record/645951/files/A_RES_2605%28XXIV%29%5BB%5D-EN.pdf
http://digitallibrary.un.org/record/509713/files/A_RES_58_278-EN.pdf
http://digitallibrary.un.org/record/41484/files/A_RES_37_107-EN.pdf
http://digitallibrary.un.org/record/748364/files/A_RES_67_182-EN.pdf
http://digitallibrary.un.org/record/188356/files/A_RES_32_63-EN.pdf
http://digitallibrary.un.org/record/815572/files/A_RES_70_88-EN.pdf
http://digitallibrary.un.org/record/203992/files/A_RES_1947%28XVIII%29-EN.pdf
http://digitallibrary.un.org/record/61099/files/A_RES_38_55-EN.pdf
http://digitallibrary.un.org/record/642692/files/A_RE

http://digitallibrary.un.org/record/190214/files/A_RES_3218%28XXIX%29-EN.pdf
http://digitallibrary.un.org/record/562615/files/A_RES_60_71-EN.pdf
http://digitallibrary.un.org/record/192769/files/A_RES_44_119%26A_RES_44_119B-EN.pdf
http://digitallibrary.un.org/record/204589/files/A_RES_1693%28XVI%29-EN.pdf
http://digitallibrary.un.org/record/660435/files/A_RES_2113%28XX%29%5EA%5E-EN.pdf
http://digitallibrary.un.org/record/666742/files/A_RES_289%28IV%29%5BA%5D-EN.pdf
http://digitallibrary.un.org/record/481245/files/A_RES_57_141-EN.pdf
http://digitallibrary.un.org/record/165835/files/A_RES_49_11-EN.pdf
http://digitallibrary.un.org/record/210574/files/A_RES_204%28III%29-EN.pdf
http://digitallibrary.un.org/record/207729/files/A_RES_1024%28XI%29-EN.pdf
http://digitallibrary.un.org/record/187840/files/A_RES_32_50-EN.pdf
http://digitallibrary.un.org/record/210090/files/A_RES_227%28III%29-EN.pdf
http://digitallibrary.un.org/record/203974/files/A_RES_1897%28XVIII%29-EN.pdf
http://digitallibrary.u

http://digitallibrary.un.org/record/180012/files/A_RES_48_183-EN.pdf
http://digitallibrary.un.org/record/158446/files/A_RES_47_122-EN.pdf
http://digitallibrary.un.org/record/538072/files/A_RES_59_221-EN.pdf
http://digitallibrary.un.org/record/562630/files/A_RES_60_85-EN.pdf
http://digitallibrary.un.org/record/613181/files/A_RES_62_56-EN.pdf
http://digitallibrary.un.org/record/3810373/files/A_RES_73_300-EN.pdf
http://digitallibrary.un.org/record/104786/files/A_RES_45_44-EN.pdf
http://digitallibrary.un.org/record/454409/files/A_RES_56_80-EN.pdf
http://digitallibrary.un.org/record/211243/files/A_RES_671%28VII%29-EN.pdf
http://digitallibrary.un.org/record/18918/files/A_RES_35_112-EN.pdf
http://digitallibrary.un.org/record/665002/files/A_RES_1843%28XVII%29%5BB%5D-EN.pdf
http://digitallibrary.un.org/record/3846396/files/A_RES_74_26-EN.pdf
http://digitallibrary.un.org/record/203194/files/A_RES_2188%28XXI%29-EN.pdf
http://digitallibrary.un.org/record/265855/files/A_RES_53_144-EN.pdf
http://dig

http://digitallibrary.un.org/record/651829/files/A_RES_2380%28XXIII%29%5BF%5D-EN.pdf
http://digitallibrary.un.org/record/509214/files/A_RES_58_138-EN.pdf
http://digitallibrary.un.org/record/588187/files/A_RES_61_40-EN.pdf
http://digitallibrary.un.org/record/747197/files/A_RES_67_230-EN.pdf
http://digitallibrary.un.org/record/1481850/files/A_RES_72_262B-EN.pdf
http://digitallibrary.un.org/record/187194/files/A_RES_33_125-EN.pdf
http://digitallibrary.un.org/record/135686/files/A_RES_46_51-EN.pdf
http://digitallibrary.un.org/record/166805/files/A_RES_49_78-EN.pdf
http://digitallibrary.un.org/record/763554/files/A_RES_68_76-EN.pdf
http://digitallibrary.un.org/record/210797/files/A_RES_547%28VI%29-EN.pdf
http://digitallibrary.un.org/record/479388/files/A_RES_57_39-EN.pdf
http://digitallibrary.un.org/record/613121/files/A_RES_62_39-EN.pdf
http://digitallibrary.un.org/record/209863/files/A_RES_86%28I%29-EN.pdf
http://digitallibrary.un.org/record/266102/files/A_RES_53_200-EN.pdf
http://digital

...

http://digitallibrary.un.org/record/613126/files/A_RES_62_42-EN.pdf
http://digitallibrary.un.org/record/505463/files/A_RES_58_10-EN.pdf
http://digitallibrary.un.org/record/1656552/files/A_RES_73_95-EN.pdf
http://digitallibrary.un.org/record/702239/files/A_RES_65_213-EN.pdf
http://digitallibrary.un.org/record/764468/files/A_RES_68_228-EN.pdf
http://digitallibrary.un.org/record/201594/files/A_RES_2781%28XXVI%29-EN.pdf
http://digitallibrary.un.org/record/618604/files/A_RES_33_71%5BB%5D-EN.pdf
http://digitallibrary.un.org/record/1656556/files/A_RES_73_98-EN.pdf
http://digitallibrary.un.org/record/191225/files/A_RES_3138%28XXVIII%29-EN.pdf
http://digitallibrary.un.org/record/673667/files/A_RES_64_178-EN.pdf
http://digitallibrary.un.org/record/136115/files/A_RES_46_205-EN.pdf
http://digitallibrary.un.org/record/698316/files/A_RES_65_125-EN.pdf
http://digitallibrary.un.org/record/1469717/files/A_RES_72_129-EN.pdf
http://digitallibrary.un.org/record/210433/files/A_RES_352%28IV%29-EN.pdf
http:/

http://digitallibrary.un.org/record/103952/files/A_RES_45_63-EN.pdf
http://digitallibrary.un.org/record/27620/files/A_RES_36_26-EN.pdf
http://digitallibrary.un.org/record/720672/files/A_RES_66_89%5BB-VII%5D-EN.pdf
http://digitallibrary.un.org/record/708264/files/A_RES_65_273-EN.pdf
http://digitallibrary.un.org/record/191219/files/A_RES_3070%28XXVIII%29-EN.pdf
http://digitallibrary.un.org/record/53868/files/A_RES_43_216-EN.pdf
http://digitallibrary.un.org/record/232649/files/A_RES_51_178-EN.pdf
http://digitallibrary.un.org/record/3848626/files/A_RES_74_150-EN.pdf
http://digitallibrary.un.org/record/629849/files/A_RES_62_251-EN.pdf
http://digitallibrary.un.org/record/204344/files/A_RES_1782%28XVII%29-EN.pdf
http://digitallibrary.un.org/record/762158/files/A_RES_68_23-EN.pdf
http://digitallibrary.un.org/record/173251/files/A_RES_49_47-EN.pdf
http://digitallibrary.un.org/record/197406/files/A_RES_48_79-EN.pdf
http://digitallibrary.un.org/record/643195/files/A_RES_63_131-EN.pdf
http://digit

http://digitallibrary.un.org/record/563172/files/A_RES_60_70-EN.pdf
http://digitallibrary.un.org/record/601878/files/A_RES_61_269-EN.pdf
http://digitallibrary.un.org/record/765627/files/A_RES_68_202-EN.pdf
http://digitallibrary.un.org/record/482290/files/A_RES_57_299-EN.pdf
http://digitallibrary.un.org/record/188375/files/A_RES_32_129-EN.pdf
http://digitallibrary.un.org/record/105691/files/A_RES_45_207-EN.pdf
http://digitallibrary.un.org/record/179565/files/A_RES_48_63-EN.pdf
http://digitallibrary.un.org/record/613080/files/A_RES_62_19-EN.pdf
http://digitallibrary.un.org/record/743874/files/A_RES_67_13-EN.pdf
http://digitallibrary.un.org/record/815292/files/A_RES_70_48-EN.pdf
http://digitallibrary.un.org/record/194378/files/A_RES_48_240B-EN.pdf
http://digitallibrary.un.org/record/265421/files/A_RES_53_78%5EB%5E-EN.pdf
http://digitallibrary.un.org/record/643182/files/A_RES_63_118-EN.pdf
http://digitallibrary.un.org/record/666163/files/A_RES_620%28VII%29%5BE%5D-EN.pdf
http://digitallibra

http://digitallibrary.un.org/record/258294/files/A_RES_52_237-EN.pdf
http://digitallibrary.un.org/record/538126/files/A_RES_59_251-EN.pdf
http://digitallibrary.un.org/record/209564/files/A_RES_404%28V%29-EN.pdf
http://digitallibrary.un.org/record/11280/files/A_RES_34_207-EN.pdf
http://digitallibrary.un.org/record/202503/files/A_RES_50_24-EN.pdf
http://digitallibrary.un.org/record/509224/files/A_RES_58_142-EN.pdf
http://digitallibrary.un.org/record/741710/files/A_RES_67_47-EN.pdf
http://digitallibrary.un.org/record/3893798/files/A_RES_75_14-EN.pdf
http://digitallibrary.un.org/record/155060/files/A_RES_47_5-EN.pdf
http://digitallibrary.un.org/record/135851/files/A_RES_46_119-EN.pdf
http://digitallibrary.un.org/record/614200/files/A_RES_62_101-EN.pdf
http://digitallibrary.un.org/record/651989/files/A_RES_2480%28XXIII%29%5BA%5D-EN.pdf
http://digitallibrary.un.org/record/613186/files/A_RES_62_61-EN.pdf
http://digitallibrary.un.org/record/492664/files/A_RES_57_283B-EN.pdf
http://digitallibra

KeyboardInterrupt: 

In [447]:
raw_small = raw.sample(n=10)

In [448]:
for i, k in raw_small.iterrows():
    if k['Text'] == '':
        url = k['url']
        req = requests.get(url)
        pages = pdf2image.convert_from_bytes(req.content)
        res = ''
        for page in pages:
            res += pytesseract.image_to_string(page) 
        k['Text'] = res

In [438]:
raw_small

,Resolution,url,Text
11296,A/RES/54/254,http://digitallibrary.un.org/record/409793/fil...,UNITED ...
12416,A/RES/64/268,http://digitallibrary.un.org/record/685934/fil...,United Nations ...
11924,A/RES/54/237[B],http://digitallibrary.un.org/record/405245/fil...,UNITED ...
15561,A/RES/40/215,http://digitallibrary.un.org/record/109485/fil...,¥. Resolutions adopted on the reports of the S...
9257,A/RES/55/225A,http://digitallibrary.un.org/record/430516/fil...,United Nations ...
757,A/RES/846(IX),http://digitallibrary.un.org/record/211977/fil...,3% General Assembly — Ninth Session\n\nthe rep...
8307,A/RES/35/171,https://digitallibrary.un.org/record/19581/fil...,Vi\n\n \n\ndamental freedoms without distincti...
4422,A/RES/52/106,http://digitallibrary.un.org/record/251479/fil...,UNITED ...
1715,A/RES/72/304,http://digitallibrary.un.org/record/1634108/fi...,United Nations ...
16317,A/RES/2788(XXVI),http://digitallibrary.un.org/record/192111/fil...,Resolutions adopted on the reports of the Thir...
